<p align="center">
  <img src="techno_volcano.png" />
</p>


In [ ]:
import sys
from kubernetes import client, config
import yaml
import requests
import uuid
from datetime import datetime

def generate_unique_job_name(base_name):
    # Example of using a UUID
    unique_id = uuid.uuid4().hex[:3]
    # Example of using a timestamp
    timestamp = datetime.now().strftime("%Y%m%d%H%M")
    # Combine the base name with a unique identifier
    unique_job_name = f"{base_name}-{unique_id}-{timestamp}"
    return unique_job_name


def get_yaml_from_git(repo_url,pvc_name):
    response = requests.get(repo_url)
    if response.status_code == 200:
        yaml_content = yaml.safe_load(response.text)
        
        # Assuming the structure of the YAML is consistent with your example,
        # update PVC names for both Master and Worker
        for role in ['Master', 'Worker']:
            if role in yaml_content['spec']['xgbReplicaSpecs']:
                for volume in yaml_content['spec']['xgbReplicaSpecs'][role]['template']['spec']['volumes']:
                    if volume['name'] == 'task-pv-storage':
                        volume['persistentVolumeClaim']['claimName'] = pvc_name    
        return yaml_content
    else:
        raise Exception(f"Failed to get YAML file from {repo_url}. Status code: {response.status_code}")
        sys.exit(1)

def get_namespace():
    # Path to the namespace file inside a pod
    namespace_path = '/var/run/secrets/kubernetes.io/serviceaccount/namespace'
    with open(namespace_path, 'r') as file:
        return file.read().strip()

def apply_custom_resource_conditionally(api_instance, group, version, namespace, plural, body):
    body['metadata']['name'] = generate_unique_job_name( body['metadata']['name']) 
    print( body['metadata']['name'])
    try:
        api_response = api_instance.create_namespaced_custom_object(
            group=group,
            version=version,
            namespace=namespace,
            plural=plural,
            body=body
        )
        print("Custom resource applied. Status: %s" % api_response)
    except client.exceptions.ApiException as e:
        print(f"Failed to apply {name}: {e}")
        sys.exit(1)

# Example usage
if __name__ == "__main__":
    config.load_incluster_config()
    api_instance = client.CustomObjectsApi()
    yaml_file_url = 'https://raw.githubusercontent.com/chasecadet/kubeflow_course/main/kubeflow_integrations/volcano_iris_train.yml'
    crd_manifest = get_yaml_from_git(yaml_file_url,"xgboostlocal")
    group = 'kubeflow.org'
    version = 'v1'
    plural = 'xgboostjobs'
    # Get namespace
    namespace = get_namespace()
    apply_custom_resource_conditionally(api_instance, group, version, namespace, plural, crd_manifest)

